# アシスタント(Assistants)

[アシスタント(assistant)](https://langchain-ai.github.io/langgraph/concepts/assistants/#resources) は、開発者が実験用にエージェントを素早く簡単に修正したり、バージョンアップしたりする方法を提供します。

## グラフに設定を提供

この `task_maistro` グラフは既にアシスタントを使うように設定されています！ 注: マエストロ(伊: 達人): maestro ですが、それをもじってわざと mAIstro としています

グラフには `configuration.py` ファイルが定義され、読み込まれています。

設定可能なフィールド(`user_id`, `todo_category`, `task_maistro_role`)にはグラフのノード内でアクセスします。

## アシスタントの作成

さて、これまで作ってきた `task_maistro` アプリでアシスタントを使う実用的なユースケースは何でしょうか？

私にとっては、異なるカテゴリのタスクに対して別々の ToDo リストを持つ機能です。

例えば、個人的なタスクにはアシスタントを、仕事のタスクには別のアシスタントを使いたいのです。

これらは`todo_category`と`task_maistro_role`という設定フィールドを使って簡単に設定できます。

![Screenshot 2024-11-18 at 9.35.55 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/673d50597f4e9eae9abf4869_Screenshot%202024-11-19%20at%206.57.01%E2%80%AFPM.png)


In [ ]:
%%capture --no-stderr
%pip install -U langgraph_sdk langgraph-cli

訳注: ひとつ下のセルを実行するために必要な準備

```bash
cd module-6/deployment
```

```bash
cp docker-compose-example.yml docker-compose.yml
```

docker-compose.yml の `OPENAI_API_KEY` と `LANGSMITH_API_KEY` を設定してください。

注: 以下のコマンドを実行するには、あらかじめ [langgraph-cli](https://pypi.org/project/langgraph-cli/) のインストールしておいてください

```bash
langgraph build -t my-image
docker compose up
```


これはグラフをデプロイしたときに作成したデフォルトのアシスタントです。


In [1]:
from langgraph_sdk import get_client

url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

### Personal assistant

これは、私が個人的なタスクを管理するために使用するパーソナルアシスタントです。


In [2]:
import json

personal_assistant = await client.assistants.create(
    # "task_maistro" はデプロイしたグラフの名前です。
    graph_id="task_maistro",
    config={"configurable": {"todo_category": "personal"}},
)
print(json.dumps(personal_assistant, indent=4, ensure_ascii=False))

{
    "assistant_id": "a3a7a7c7-98ff-4bc8-8639-109995483596",
    "graph_id": "task_maistro",
    "created_at": "2024-11-24T07:39:12.961883+00:00",
    "updated_at": "2024-11-24T07:39:12.961883+00:00",
    "config": {
        "configurable": {
            "todo_category": "personal"
        }
    },
    "metadata": {},
    "version": 1,
    "name": "Untitled"
}


私の`user_id`を含むようにこのアシスタントを更新して、[新しいバージョンを作成](https://langchain-ai.github.io/langgraph/cloud/how-tos/assistant_versioning/#create-a-new-version-for-your-assistant)しましょう。


In [3]:
task_maistro_role = """\
あなたはフレンドリーで組織的な個人タスクアシスタントです。あなたの主な仕事は、ユーザーが個人的なタスクやコミットメントを管理できるようにすることです。
具体的には:

- 個人的なタスクの追跡と整理を支援する
- 「Todo サマリー」を提供する場合:
    1. 期限（期限切れ、今日、今週、将来）ごとにグループ化された現在のタスクをすべてリストアップする
    2. 期限を過ぎているタスクを強調し、追加するようやさしく勧める
    3. 重要だと思われるタスクでも、見積もり時間が不足している場合は注意してください。
- 重要と思われるが、時間の見積もりがないタスクが追加されたら、積極的に期限を尋ねる
- ユーザーの説明責任を果たせるようにサポートしながら、サポート的な口調を保つ
- 期限と重要性に基づいてタスクの優先順位を決める手助けをする

あなたのコミュニケーションスタイルは、励ましと手助けであるべきで、決して批判的であってはいけません。

タスクに期限がない場合は、「[タスク]にはまだ期限がありませんね。タスクの進捗状況を把握しやすくするために、期限を追加しますか？」 のように返答してください。
タスクの追跡を向上させるために、追加しますか？"""

configurations = {
    "todo_category": "personal",
    "user_id": "lance",
    "task_maistro_role": task_maistro_role,
}

personal_assistant = await client.assistants.update(
    assistant_id=personal_assistant["assistant_id"], config={"configurable": configurations}
)
print(json.dumps(personal_assistant, indent=4, ensure_ascii=False))

{
    "assistant_id": "a3a7a7c7-98ff-4bc8-8639-109995483596",
    "graph_id": "task_maistro",
    "created_at": "2024-11-24T07:39:16.328286+00:00",
    "updated_at": "2024-11-24T07:39:16.328286+00:00",
    "config": {
        "configurable": {
            "user_id": "lance",
            "todo_category": "personal",
            "task_maistro_role": "あなたはフレンドリーで組織的な個人タスクアシスタントです。あなたの主な仕事は、ユーザーが個人的なタスクやコミットメントを管理できるようにすることです。\n具体的には:\n\n- 個人的なタスクの追跡と整理を支援する\n- 「Todo サマリー」を提供する場合:\n    1. 期限（期限切れ、今日、今週、将来）ごとにグループ化された現在のタスクをすべてリストアップする\n    2. 期限を過ぎているタスクを強調し、追加するようやさしく勧める\n    3. 重要だと思われるタスクでも、見積もり時間が不足している場合は注意してください。\n- 重要と思われるが、時間の見積もりがないタスクが追加されたら、積極的に期限を尋ねる\n- ユーザーの説明責任を果たせるようにサポートしながら、サポート的な口調を保つ\n- 期限と重要性に基づいてタスクの優先順位を決める手助けをする\n\nあなたのコミュニケーションスタイルは、励ましと手助けであるべきで、決して批判的であってはいけません。\n\nタスクに期限がない場合は、「[タスク]にはまだ期限がありませんね。タスクの進捗状況を把握しやすくするために、期限を追加しますか？」 のように返答してください。\nタスクの追跡を向上させるために、追加しますか？"
        }
    },
    "metadata": {},
    "version": 2,
    "name": "Untitled"
}


### Work assistant

では、ワークアシスタントを作りましょう。これを仕事のタスクに使います。


In [4]:
task_maistro_role = """\
あなたは、集中して効率的に作業タスクを支援するアシスタントです。

あなたの主な焦点は、ユーザーが現実的な時間枠で仕事のコミットメントを管理するのを支援することです。

具体的には:

- 仕事のタスクの追跡と整理を支援する
- "todo summary" を提供するとき:
    1. 現在のすべてのタスクを期限別にグループ化してリストにする（期限切れ、本日、今週、将来）
    2. 期限が設定されていないタスクを強調し、優しく追加を促す
    3. 重要なタスクで時間見積もりが不足しているものを指摘する
- 新しいタスクについて議論するときは、タスクのタイプに応じた現実的な時間枠を提示するようにユーザーに提案しましょう:
    - ディベロッパーリレーションズの機能: 通常1日
    - コースレッスンのレビュー/フィードバック: 通常2日
    - ドキュメントスプリント: 通常3日
- 期限とチーム依存関係に基づいてタスクの優先順位を手助けする
- ユーザーが責任を持つよう支援しつつ、プロフェッショナルな口調を保つ

あなたのコミュニケーションスタイルは、支援的でありながら実用的であるべきです。

タスクに期限が設定されていない場合、「[タスク]にはまだ期限が設定されていないことに気付きました。類似のタスクに基づくと、これには[提案された時間枠]がかかるかもしれません。このことを考慮して期限を設定しますか？」のように返答します。
"""

configurations = {
    "todo_category": "work",
    "user_id": "lance",
    "task_maistro_role": task_maistro_role,
}

work_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    graph_id="task_maistro",
    config={"configurable": configurations},
)

print(json.dumps(work_assistant, indent=4, ensure_ascii=False))

{
    "assistant_id": "93355ff8-de80-4935-8fe8-490ca4f89595",
    "graph_id": "task_maistro",
    "created_at": "2024-11-24T07:39:25.841333+00:00",
    "updated_at": "2024-11-24T07:39:25.841333+00:00",
    "config": {
        "configurable": {
            "user_id": "lance",
            "todo_category": "work",
            "task_maistro_role": "あなたは、集中して効率的に作業タスクを支援するアシスタントです。\n\nあなたの主な焦点は、ユーザーが現実的な時間枠で仕事のコミットメントを管理するのを支援することです。\n\n具体的には:\n\n- 仕事のタスクの追跡と整理を支援する\n- \"todo summary\" を提供するとき:\n    1. 現在のすべてのタスクを期限別にグループ化してリストにする（期限切れ、本日、今週、将来）\n    2. 期限が設定されていないタスクを強調し、優しく追加を促す\n    3. 重要なタスクで時間見積もりが不足しているものを指摘する\n- 新しいタスクについて議論するときは、タスクのタイプに応じた現実的な時間枠を提示するようにユーザーに提案しましょう:\n    - ディベロッパーリレーションズの機能: 通常1日\n    - コースレッスンのレビュー/フィードバック: 通常2日\n    - ドキュメントスプリント: 通常3日\n- 期限とチーム依存関係に基づいてタスクの優先順位を手助けする\n- ユーザーが責任を持つよう支援しつつ、プロフェッショナルな口調を保つ\n\nあなたのコミュニケーションスタイルは、支援的でありながら実用的であるべきです。\n\nタスクに期限が設定されていない場合、「[タスク]にはまだ期限が設定されていないことに気付きました。類似のタスクに基づくと、これには[提案された時間枠]がかかるかもしれません。このことを考慮して期限を設定しますか？」のように返答し

## Using assistants

アシスタントはデプロイ時に`Postgres`に保存されます。

これにより、SDK でアシスタントを簡単に検索[search](https://langchain-ai.github.io/langgraph/cloud/how-tos/configuration_cloud/)することができます。


In [5]:
assistants = await client.assistants.search()
for assistant in assistants:
    print(
        json.dumps(
            {"assistant_id": assistant["assistant_id"], "version": assistant["version"], "config": assistant["config"]},
            indent=4,
            ensure_ascii=False,
        )
    )


{
    "assistant_id": "93355ff8-de80-4935-8fe8-490ca4f89595",
    "version": 1,
    "config": {
        "configurable": {
            "user_id": "lance",
            "todo_category": "work",
            "task_maistro_role": "あなたは、集中して効率的に作業タスクを支援するアシスタントです。\n\nあなたの主な焦点は、ユーザーが現実的な時間枠で仕事のコミットメントを管理するのを支援することです。\n\n具体的には:\n\n- 仕事のタスクの追跡と整理を支援する\n- \"todo summary\" を提供するとき:\n    1. 現在のすべてのタスクを期限別にグループ化してリストにする（期限切れ、本日、今週、将来）\n    2. 期限が設定されていないタスクを強調し、優しく追加を促す\n    3. 重要なタスクで時間見積もりが不足しているものを指摘する\n- 新しいタスクについて議論するときは、タスクのタイプに応じた現実的な時間枠を提示するようにユーザーに提案しましょう:\n    - ディベロッパーリレーションズの機能: 通常1日\n    - コースレッスンのレビュー/フィードバック: 通常2日\n    - ドキュメントスプリント: 通常3日\n- 期限とチーム依存関係に基づいてタスクの優先順位を手助けする\n- ユーザーが責任を持つよう支援しつつ、プロフェッショナルな口調を保つ\n\nあなたのコミュニケーションスタイルは、支援的でありながら実用的であるべきです。\n\nタスクに期限が設定されていない場合、「[タスク]にはまだ期限が設定されていないことに気付きました。類似のタスクに基づくと、これには[提案された時間枠]がかかるかもしれません。このことを考慮して期限を設定しますか？」のように返答します。\n"
        }
    }
}
{
    "assistant_id": "a3a7a7c7-98ff-4bc8-8639-109995483596",
    "version": 2,
    "config": {
 

SDK で簡単に管理できます。例えば、使わなくなったアシスタントを削除することができます。


In [6]:
# このIDは上記で実行したIDを使ってください。
# このアシスタントは後で使うので、消すのはやめておいてください。
# your_assistant_id ="a25dcb41-bebb-41d5-8957-70567ae87817" # 例です。
# await client.assistants.delete(assistant_id=your_assistant_id)


`個人用`と`仕事用`のアシスタント ID を設定してみましょう。


In [7]:
work_assistant_id = assistants[0]["assistant_id"]
personal_assistant_id = assistants[1]["assistant_id"]

### Work assistant

仕事用のアシスタントに ToDo を追加しましょう。


In [8]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import convert_to_messages

user_input = "いくつかのToDoを作成または更新します:  1) 今日中にモジュール6、レッスン5を再撮影。2) 来週月曜日までにオーディオUXを更新。"
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread_id=thread["thread_id"],
    assistant_id=work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(messages=state["messages"])[-1].pretty_print()

================================ Human Message =================================

いくつかのToDoを作成または更新します:  1) 今日中にモジュール6、レッスン5を再撮影。2) 来週月曜日までにオーディオUXを更新。


In [13]:
user_input = "もうひとつToDoを作成してください: レポート生成チュートリアル集を仕上げる"
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread_id=thread["thread_id"],
    assistant_id=work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(messages=state["messages"])[-1].pretty_print()

================================ Human Message =================================

もうひとつToDoを作成してください: レポート生成チュートリアル集を仕上げる


訳注: assistant.ipynb を見ると、 Tool Calls UpdateMemory が呼ばれているが、各種プロンプトを日本語で書いたためか呼ばれていない


アシスタントの指示に従い、タスクの作成を進めます！

期限を指定してください


In [14]:
user_input = "よし、この仕事は来週の火曜日までに終わらせましょう。"
async for chunk in client.runs.stream(
    thread_id=thread["thread_id"],
    assistant_id=work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(messages=state["messages"])[-1].pretty_print()

================================ Human Message =================================

よし、この仕事は来週の火曜日までに終わらせましょう。


### Personal assistant

同様に、パーソナルアシスタントにも ToDo を追加できます


In [15]:
user_input = (
    "ToDoを作成します:  1) 今週末に赤ちゃんの水泳教室をチェック。2) 冬の旅行のために、アメックスのポイントをチェック。"
)
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread_id=thread["thread_id"],
    assistant_id=personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

ToDoを作成します:  1) 今週末に赤ちゃんの水泳教室をチェック。2) 冬の旅行のために、アメックスのポイントをチェック。


In [16]:
user_input = "ToDoのまとめを教えてください。"
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread_id=thread["thread_id"],
    assistant_id=personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):
    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

ToDoのまとめを教えてください。
